In [1]:
#for local
import sys
repo_root = "../"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

In [2]:
from datetime import datetime
import importlib
import numpy as np
import logging
import os
import shutil
from typing import Any
import yaml
import optuna
from rdkit import RDLogger
from generator import Generator
from language import Language
from node import MolSentenceNode
from utils import add_sep, class_from_package, make_logger
RDLogger.DisableLog('rdApp.*')

yaml_path = "config/optuna_generation.yaml"

with open(os.path.join(repo_root, yaml_path)) as f:
    conf = yaml.safe_load(f)
transition_args = conf.get("transition_args", {})
model_dir = os.path.join(repo_root, transition_args.pop("model_dir"))
lang_path = conf.get("lang_path")
if lang_path is None:
    lang_name = os.path.basename(os.path.normpath(model_dir)) + ".lang"
    lang_path = add_sep(model_dir) + lang_name
lang = Language.load(lang_path)
policy_args = conf.get("policy_args", {})
reward_class = class_from_package("reward", conf.get("reward_class"))
reward = reward_class(**conf.get("reward_args", {}))
filter_settings = conf.get("filters", [])
filters = []
for s in filter_settings:
    filter_class = class_from_package("filter", s.pop("filter_class"))
    filters.append(filter_class(**s))
generator_args = conf.get("generator_args", {})

def objective(trial):
    transition_args["top_p"] = 1 - trial.suggest_loguniform("1-top_p", 0.003, 0.007)
    transition_args["power"] = trial.suggest_uniform("power", 0.7, 1.3)
    # transition_args["temperature"] = trial.suggest_uniform("temperature", 0.7, 1.2)
    # policy_class = trial.suggest_categorical("policy_class", ["UCB", "PUCT"])
    policy_class = "UCB1"
    policy_args["c"] = trial.suggest_uniform("c", 0.01, 1)
    policy_args["best_rate"] = trial.suggest_uniform("best_rate", 0, 1)
    # generator_args["filtered_reward"] = trial.suggest_uniform("filtered_reward", -1, 0.2)
    generator_args["rollout_width"] = trial.suggest_int("rollout_width", 1, 40)
    # generator_args["allow_rollout_overlaps"] = trial.suggest_categorical("allow_rollout_overlaps", [True, False])
    generator_args["n_rollouts"] = trial.suggest_int("n_rollouts", 1, 10)
    generator_args["n_tries"] = trial.suggest_int("n_tries", 1, 3)
    generator_args["terminal_reward"] = trial.suggest_categorical("terminal_reward", ["ignore", -1])
    generator_args["remove_failed_child"] = trial.suggest_categorical("remove_failed_child", [True, False])

    output_dir=os.path.join(repo_root, "sandbox", conf["output_dir"], datetime.now().strftime("%m-%d_%H-%M")) + os.sep
    console_level = logging.ERROR
    file_level = logging.DEBUG if conf.get("debug") else logging.INFO
    logger = make_logger(output_dir, console_level=console_level, file_level=file_level)
    logger.info("params:" + str(trial.params))

    transition_class = class_from_package("transition", conf["transition_class"])
    transition = transition_class(model_dir=model_dir, lang=lang, logger=logger, device=conf.get("device"), **transition_args)
    
    policy_class = class_from_package("policy", policy_class)
    policy = policy_class(**policy_args)
    generator_args["policy"] = policy
        
    root = MolSentenceNode.bos_node(lang, device=conf.get("device")) # TODO: change after root node generalization
    
    generator_class = class_from_package("generator", conf.get("generator_class", "MCTS"))
    generator = generator_class(root=root, transition=transition, reward=reward, filters=filters, output_dir=output_dir, logger=logger, **generator_args)
    
    max_generations, time_limit = conf.get("max_generations"), conf.get("time_limit")
    n_steps, best_reward_rate, recent_mean_window = conf.get("n_steps"), conf.get("best_reward_rate"), conf.get("recent_mean_window")
    
    for i in range(0, n_steps):
        generator.generate(max_generations=max_generations / n_steps, time_limit=time_limit / n_steps)
        mean_reward = generator.mean_reward()
        recent_mean_reward = generator.mean_reward(window=recent_mean_window)
        intermediate_value = (1 - best_reward_rate) * mean_reward + best_reward_rate * generator.best_reward
        trial.report(intermediate_value, i)
        if trial.should_prune():
            print(f"Trial {trial.number} - Step {i}: best_reward={generator.best_reward:.3f}, params={trial.params}, mean_reward={mean_reward:.3f}")
            raise optuna.TrialPruned()

    trial.set_user_attr("best_reward", generator.best_reward)
    trial.set_user_attr("recent_mean_reward", recent_mean_reward)
    trial.set_user_attr("mean_reward", mean_reward)
    print(f"Trial {trial.number}: best_reward={generator.best_reward:.3f}, recent_mean_reward={recent_mean_reward:.3f}, mean_reward={mean_reward:.3f}")
    generator.analyze()
    generator.plot(**conf.get("plot_args"))
    return (1 - best_reward_rate) * mean_reward + best_reward_rate * generator.best_reward
    
def print_trial(trial: optuna.Trial):
    print(f"Trial {trial.number} score={trial.value:.3f}, mean_reward={trial.user_attrs['mean_reward']:.3f}, best_reward={trial.user_attrs['best_reward']:.3f}, params={trial.params}")
    
def print_best_trials(study: optuna.Study):
    print("Optuna trials completed.")
    print("------ Best trials -----")
    best_trials = sorted([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE], key=lambda t: t.value, reverse=True)[:5]
    for t in best_trials:
        print_trial(t)

/home/fujii/anaconda3/envs/v3-forge/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.


In [3]:
# start search
name = conf.get("study_name")
storage = "sqlite:///optuna/" + name + ".db"
sampler = sampler=optuna.samplers.TPESampler(multivariate=True, group=True)
# sampler = optuna.samplers.GPSampler(deterministic_objective=False) # better if not using pruner?
pruner = optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=0, interval_steps=1)
study = optuna.create_study(direction="maximize", study_name=name, storage=storage, sampler=sampler, pruner=pruner)
study.enqueue_trial({"1-top_p": 0.005, "power": 1.0, "c": 0.2, "best_rate": 0.5, "rollout_width": 1, "n_rollouts": 1, "n_tries": 1, "terminal_reward": "ignore", "remove_failed_child": False})
study.enqueue_trial({"1-top_p": 0.005, "power": 1.0, "c": 0.2, "best_rate": 0.5, "rollout_width": 3, "n_rollouts": 3, "n_tries": 1, "terminal_reward": -1, "remove_failed_child": True})
study.enqueue_trial({"1-top_p": 0.005, "power": 1.2, "c": 0.2, "best_rate": 0.5, "rollout_width": 9, "n_rollouts": 1, "n_tries": 1, "terminal_reward": -1, "remove_failed_child": False})
study.optimize(objective, n_trials=conf.get("n_trials"))
print_best_trials(study)

/home/fujii/anaconda3/envs/v3-forge/lib/python3.11/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
/home/fujii/anaconda3/envs/v3-forge/lib/python3.11/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``group`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-06-24 14:54:19,331] A new study created in RDB with name: d_score_200000


Trial 0: best_reward=0.589, recent_mean_reward=0.398, mean_reward=0.395


[I 2025-06-24 16:39:23,790] Trial 0 finished with value: 0.4532229667300164 and parameters: {'1-top_p': 0.005, 'power': 1.0, 'c': 0.2, 'best_rate': 0.5, 'rollout_width': 1, 'n_rollouts': 1, 'n_tries': 1, 'terminal_reward': 'ignore', 'remove_failed_child': False}. Best is trial 0 with value: 0.4532229667300164.
[W 2025-06-24 16:39:24,884] Trial 1 failed with parameters: {'1-top_p': 0.005, 'power': 1.0, 'c': 0.2, 'best_rate': 0.5, 'rollout_width': 3, 'n_rollouts': 3, 'n_tries': 1, 'terminal_reward': -1, 'remove_failed_child': True} because of the following error: KeyError(15).
Traceback (most recent call last):
  File "/home/fujii/anaconda3/envs/v3-forge/lib/python3.11/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_1849575/3182275023.py", line 76, in objective
    generator.generate(max_generations=max_generations / n_steps, time_limit=time_limit / n_steps)
  File "/home/fujii/mo

KeyError: 15

In [ ]:
# continue search
study = optuna.study.load_study(study_name="d_score_200000", storage="sqlite:///optuna/d_score_200000.db")
study.optimize(objective, n_trials=300)
print_best_trials(study)

In [ ]:
# add parameters
study = optuna.study.load_study(study_name="d_score_200000", storage="sqlite:///sqlite:///optuna/d_score_200000.db")

name = conf.get("study_name")
sampler = sampler=optuna.samplers.TPESampler(multivariate=True, group=True)
pruner = optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=0, interval_steps=1)
new_storage="sqlite:///optuna/d_score_200000_new.db"
study_with_new_param = optuna.create_study(direction="maximize", study_name=name, storage=new_storage, sampler=sampler, pruner=pruner)

for trial in study.trials:
    params = trial.params
    dists = trial.distributions

    params["remove_failed_child"] = False
    dists["remove_failed_child"] = optuna.distributions.CategoricalDistribution([True, False])

    trial.params = params
    trial.distributions = dists

    study_with_new_param.add_trial(trial)